## LOST
* https://github.com/camelot-dev/camelot
* https://stackoverflow.com/questions/62044535/how-to-extract-tables-from-pdf-using-camelot
* https://camelot-py.readthedocs.io/en/master/user/quickstart.html#specify-page-numbers

* pip install "camelot-py[base]"

In [1]:
import camelot
import _utils
import pandas as pd
from calitp_data_analysis.sql import to_snakecase

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/project_list/LOST/"

In [4]:
alameda_b_2000 = _utils.open_pdf("alameda_b_2000", "41,42")

In [5]:
los_angeles_m_2016 = _utils.open_pdf("los_angeles_m_2016", "25,26")

In [6]:
monterey_x_2016 = _utils.open_pdf("monterey_x_2016", "3")

In [7]:
san_bernardino_I2_2018 = _utils.open_pdf("san_bernardino_I2_2018", "11,12,13,14,15,16,17,18,19,20")

In [8]:
san_joaquin_k_2003 = _utils.open_pdf("san_joaquin_k_2003", "30,31,32,33")

/opt/conda/lib/python3.9/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")
/opt/conda/lib/python3.9/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 2
  warnings.warn(f"No tables found in table area {table_idx + 1}")


In [9]:
tulare_r_2006 = _utils.open_pdf("tulare_r_2006", "13,14,15,16,17")

In [10]:
santa_clara_a_2000 = _utils.open_pdf("santa_clara_a_2000", "1")

In [11]:
alameda_b_2000= _utils.clean_LOST_pdf(alameda_b_2000, 5, "alameda_b_2000", "")

In [12]:
los_angeles_m_2016 = _utils.clean_LOST_pdf(los_angeles_m_2016, 11, "los_angeles_m_2016", "")

In [13]:
with pd.ExcelWriter(f"{GCS_FILE_PATH}LOST.xlsx") as writer:
    alameda_b_2000.to_excel(writer, sheet_name="alameda_b_2000", index=True)
    los_angeles_m_2016.to_excel(writer, sheet_name="los_angeles_m_2016", index=True)
    monterey_x_2016.to_excel(writer, sheet_name="monterey_x_2016", index=True)
    san_bernardino_I2_2018.to_excel(writer, sheet_name="san_bernardino_I2_2018", index=True)
    san_joaquin_k_2003.to_excel(writer, sheet_name="san_joaquin_k_2003", index=True)
    santa_clara_a_2000.to_excel(writer, sheet_name="santa_clara_a_2000", index=True)
    tulare_r_2006.to_excel(writer, sheet_name="tulare_r_2006", index=True)